In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from datetime import datetime
from sklearn.metrics import mean_squared_error, r2_score
import shap
import seaborn as sns 
import matplotlib.pyplot as plt

# Assuming df is your DataFrame after aggregating to daily data and merging with weather/building data
df = pd.read_csv("Data.csv")# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# Get the number of rows and columns 
# rows = len(df.axes[0]) 
# cols = len(df.axes[1]) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41666 entries, 0 to 41665
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   time                               41664 non-null  object 
 1   Comms and Services                 40812 non-null  float64
 2   Car Chargers                       40812 non-null  float64
 3   Space Heating                      40812 non-null  float64
 4   Hot Water                          40812 non-null  float64
 5   Sockets                            40812 non-null  float64
 6   Lighting                           40812 non-null  float64
 7   total_energy                       41664 non-null  object 
 8   datepart                           41664 non-null  float64
 9   weekend                            41664 non-null  object 
 10  bank holiday                       41664 non-null  object 
 11  hour                               41664 non-null  flo

C:\Users\mscVRstudent22\AppData\Local\Temp\ipykernel_24924\267611544.py:17: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Data.csv")# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"


In [12]:
# Generate a list of columns to drop
columns_to_drop = [col for col in df.columns if col.startswith('observation')]

# Drop these columns from the DataFrame
df = df.drop(columns=columns_to_drop)
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40131 entries, 112 to 41663
Data columns (total 27 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   time                               40131 non-null  object 
 1   Comms and Services                 40131 non-null  float64
 2   Car Chargers                       40131 non-null  float64
 3   Space Heating                      40131 non-null  float64
 4   Hot Water                          40131 non-null  float64
 5   Sockets                            40131 non-null  float64
 6   Lighting                           40131 non-null  float64
 7   total_energy                       40131 non-null  object 
 8   datepart                           40131 non-null  float64
 9   weekend                            40131 non-null  object 
 10  bank holiday                       40131 non-null  object 
 11  hour                               40131 non-null  float6

In [ ]:
# #Defining Function for turning features into cyclic 
# def encode(data, col, max_val):
#     data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
#     data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
#     return data

In [ ]:


# # df['month'] = df.datetime.dt.month
# df = encode(df, 'month', 12)
# df =  encode(df, 'hour', 24)
# # df['day'] = df.datetime.dt.day
# # df = encode(df, 'day', 31)
# df = encode(df, 'day of week', 7)
# df = encode(df, 'day of month', 31)
# df['working_hours'] = df['hour'].apply(lambda x: 8 <= x <= 17)
# df['bank holiday'] = df['bank holiday'].astype(int)
# df['weekend'] = df['weekend'].astype(int)


In [ ]:


# def periodic_spline_transformer(period, n_splines=None, degree=3):
#     if n_splines is None:
#         n_splines = period
#     n_knots = n_splines + 1  # periodic and include_bias is True
#     return SplineTransformer(
#         degree=degree,
#         n_knots=n_knots,
#         knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
#         extrapolation="periodic",
#         include_bias=True,
#     )

In [ ]:

# # df['month'] = df.datetime.dt.month
# df= encode(df, 'month', 12)
# df =  encode(df, 'hour', 24)
# # df['day'] = df.datetime.dt.day
# # df = encode(df, 'day', 31)
# df = encode(df, 'day of week', 7)
# df = encode(df, 'day of month', 31)
# df['working_hours'] = df['hour'].apply(lambda x: 8 <= x <= 17)
# df['bank holiday'] = df['bank holiday'].astype(int)
# df['weekend'] = df['weekend'].astype(int)


In [8]:
# df.dropna(inplace =True)

In [13]:
from sklearn.preprocessing import SplineTransformer

def periodic_spline_transformer(period, n_splines=None, degree=3):
    if n_splines is None:
        n_splines = period
    n_knots = n_splines + 1  # periodic and include_bias is True
    return SplineTransformer(
        degree=degree,
        n_knots=n_knots,
        knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
        extrapolation="periodic",
        include_bias=True,
    )

# Apply Spline Transformation
def apply_spline_transformer(df, column, period):
    spline_transformer = periodic_spline_transformer(period)
    transformed_values = spline_transformer.fit_transform(df[[column]])
    
    # Create new column names for the transformed features
    transformed_cols = [f"{column}_spline_{i}" for i in range(transformed_values.shape[1])]
    df[transformed_cols] = transformed_values
    
    return df

In [15]:
# Apply spline transformation for the temporal features
df = apply_spline_transformer(df, 'month', 12)
df = apply_spline_transformer(df, 'hour', 24)
df = apply_spline_transformer(df, 'day of week', 7)
df = apply_spline_transformer(df, 'day of month', 31)

# Additional feature engineering
df['working_hours'] = df['hour'].apply(lambda x: 8 <= x <= 17)
df['bank holiday'] = df['bank holiday'].astype(int)
df['weekend'] = df['weekend'].astype(int)

# Drop the original columns if no longer needed
df.drop(columns=['month', 'hour', 'day of week', 'day of month'], inplace=True)

# Display the updated DataFrame
print(df.head())

                       time  Comms and Services  Car Chargers  Space Heating  \
112  2019-04-05 16:00:00+00            0.590111      0.043272       2.564617   
113  2019-04-05 17:00:00+00            0.590689      0.043253       2.639575   
115  2019-04-05 19:00:00+00            0.590050      0.043139       1.168772   
116  2019-04-05 20:00:00+00            0.589536      0.043031       0.916325   
119  2019-04-05 23:00:00+00            0.589953      0.043239       0.953847   

     Hot Water   Sockets  Lighting total_energy  datepart  weekend  ...  \
112   0.076094  0.077019  0.756983  4.108097218   43560.0        0  ...   
113   0.000000  0.048581  0.208511  3.530608334   43560.0        0  ...   
115   0.000000  0.052336  0.151611  2.005908333   43560.0        0  ...   
116   0.000000  0.046442  0.151839  1.747172223   43560.0        0  ...   
119   0.087725  0.067442  0.173542  1.915747223   43560.0        0  ...   

     day of month_spline_22  day of month_spline_23 day of month_spl

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40131 entries, 112 to 41663
Data columns (total 98 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   time                               40131 non-null  object 
 1   Comms and Services                 40131 non-null  float64
 2   Car Chargers                       40131 non-null  float64
 3   Space Heating                      40131 non-null  float64
 4   Hot Water                          40131 non-null  float64
 5   Sockets                            40131 non-null  float64
 6   Lighting                           40131 non-null  float64
 7   total_energy                       40131 non-null  object 
 8   datepart                           40131 non-null  float64
 9   weekend                            40131 non-null  int32  
 10  bank holiday                       40131 non-null  int32  
 11  year                               40131 non-null  float6

In [17]:
columns_to_convert = ['weekend','bank holiday']

# Convert 0 and 1 to boolean values in specified columns
df[columns_to_convert] = df[columns_to_convert].astype(bool)
# Sum specified columns to create 'total_aob_energy'
df['total_aob_energy'] = df[[ 'Hot Water', 'Sockets', 'Lighting','Comms and Services', 'Space Heating']].sum(axis=1, skipna=True)

# Drop the original columns except 'Car Chargers'
df.drop(['Comms and Services', 'Space Heating', 'Hot Water', 'Sockets', 'Lighting' ], axis=1, inplace=True)

# Now, 'df' contains the new 'total_aob_energy' column and has the specified columns dropped, except 'Car Chargers'

In [18]:
df['time'] = pd.to_datetime(df['time'])

In [20]:
df = df.dropna()
df = df.select_dtypes(exclude=['object'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40131 entries, 112 to 41663
Data columns (total 88 columns):
 #   Column                             Non-Null Count  Dtype              
---  ------                             --------------  -----              
 0   time                               40131 non-null  datetime64[ns, UTC]
 1   Car Chargers                       40131 non-null  float64            
 2   datepart                           40131 non-null  float64            
 3   weekend                            40131 non-null  bool               
 4   bank holiday                       40131 non-null  bool               
 5   year                               40131 non-null  float64            
 6   forecast_temperature               40131 non-null  float64            
 7   forecast_feelslike                 40131 non-null  float64            
 8   forecast_weathertype               40131 non-null  float64            
 9   forecast_windspeed                 40131 non-null  fl

In [21]:
df.columns = [col.capitalize() for col in df.columns]
print(df.columns)
df.info()

Index(['Time', 'Car chargers', 'Datepart', 'Weekend', 'Bank holiday', 'Year',
       'Forecast_temperature', 'Forecast_feelslike', 'Forecast_weathertype',
       'Forecast_windspeed', 'Forecast_uvindex',
       'Forecast_precipitationprobability', 'Month_spline_0', 'Month_spline_1',
       'Month_spline_2', 'Month_spline_3', 'Month_spline_4', 'Month_spline_5',
       'Month_spline_6', 'Month_spline_7', 'Month_spline_8', 'Month_spline_9',
       'Month_spline_10', 'Month_spline_11', 'Hour_spline_0', 'Hour_spline_1',
       'Hour_spline_2', 'Hour_spline_3', 'Hour_spline_4', 'Hour_spline_5',
       'Hour_spline_6', 'Hour_spline_7', 'Hour_spline_8', 'Hour_spline_9',
       'Hour_spline_10', 'Hour_spline_11', 'Hour_spline_12', 'Hour_spline_13',
       'Hour_spline_14', 'Hour_spline_15', 'Hour_spline_16', 'Hour_spline_17',
       'Hour_spline_18', 'Hour_spline_19', 'Hour_spline_20', 'Hour_spline_21',
       'Hour_spline_22', 'Hour_spline_23', 'Day of week_spline_0',
       'Day of week_splin

In [22]:
df = df.drop(['Car chargers','Datepart'], axis = 1)

In [23]:
new_columns = []
for col in df.columns:
    # Remove "Forecast_" if it exists and capitalize the first letter of the remaining string
    if col.startswith('Forecast_'):
        new_name = col.replace('Forecast_', '')  # Remove 'Forecast_'
        new_name = new_name.capitalize()  # Capitalize the first letter
    else:
        new_name = col  # Keep the original name if it doesn't start with 'Forecast_'
    new_columns.append(new_name)

# Assign the modified column names back to the DataFrame
df.columns = new_columns


In [24]:
# Set 'date' column as the DataFrame index
df.set_index('Time', inplace=True)
#removing precovid data
df = df[df.index >= "2021-01-01"]

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 25470 entries, 2021-01-01 00:00:00+00:00 to 2023-12-31 23:00:00+00:00
Data columns (total 85 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Weekend                   25470 non-null  bool   
 1   Bank holiday              25470 non-null  bool   
 2   Year                      25470 non-null  float64
 3   Temperature               25470 non-null  float64
 4   Feelslike                 25470 non-null  float64
 5   Weathertype               25470 non-null  float64
 6   Windspeed                 25470 non-null  float64
 7   Uvindex                   25470 non-null  float64
 8   Precipitationprobability  25470 non-null  float64
 9   Month_spline_0            25470 non-null  float64
 10  Month_spline_1            25470 non-null  float64
 11  Month_spline_2            25470 non-null  float64
 12  Month_spline_3            25470 non-null  float64
 13  Month_spline_4

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import shap
import mifs
from boruta import BorutaPy

In [28]:
# Remove rare/uncommon values (assuming they are below a threshold or beyond a threshold)
df = df[(df['Total_aob_energy'] > df['Total_aob_energy'].quantile(0.01)) & 
        (df['Total_aob_energy'] < df['Total_aob_energy'].quantile(0.99))]

# Define the date ranges for training and testing
train_data = df[((df.index.year == 2021) | (df.index.year == 2022)) | 
                       ((df.index.year == 2023) & (df.index.day <= 14))]
test_data = df[(df.index.year == 2023) & (df.index.day > 14)]

# Split data into features and target
X_train = train_data.drop(['Total_aob_energy'], axis=1)
y_train = train_data['Total_aob_energy']
X_test = test_data.drop(['Total_aob_energy'], axis=1)
y_test = test_data['Total_aob_energy']

# # Train and evaluate the model for Split 1
# model_split1 = LinearRegression()
# model_split1.fit(X_train_split1, y_train_split1)
# y_pred_split1 = model_split1.predict(X_test_split1)
# rmse_split1 = mean_squared_error(y_test_split1, y_pred_split1, squared=False)
# print(f"Split 1 RMSE: {rmse_split1}")

# # Verify the date ranges
# print("Split 1 - Train set:", X_train_split1.index.min(), "to", X_train_split1.index.max())
# print("Split 1 - Test set:", X_test_split1.index.min(), "to", X_test_split1.index.max())
# Split the data into training and testing sets using time series split
# tscv = TimeSeriesSplit(n_splits=5)
# tscv = TimeSeriesSplit(n_splits=2)
# for train_index, test_index in tscv.split(X):
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
# print(X_test, y_test)
# X_train = train_data.drop(['Total_aob_energy'], axis=1)
# y_train = train_data['Total_aob_energy']
# X_test = test_data.drop(['Total_aob_energy'], axis=1)
# y_test = test_data['Total_aob_energy']

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [29]:

# Feature Selection with Lasso
lasso = Lasso(alpha=0.01)
lasso.fit(X_train_scaled, y_train)
lasso_features = (lasso.coef_ != 0)
print(lasso_features)

# Feature Selection with Random Forest and Boruta
rf = RandomForestRegressor(n_jobs=-1, random_state=42)
boruta_selector = BorutaPy(rf, n_estimators='auto', random_state=42)
boruta_selector.fit(X_train_scaled, y_train)
boruta_features = boruta_selector.support_
print(boruta_features)

[ True  True  True False  True  True  True  True False False  True  True
  True False  True  True  True  True False False  True  True  True  True
 False  True  True  True False  True  True  True  True  True False False
 False False False False  True  True  True  True  True  True False  True
  True  True False False  True False False  True False  True False  True
  True  True  True  True  True  True  True  True False  True False False
  True  True  True False False  True  True False  True  True False  True]
[ True False  True  True  True False  True False  True False  True  True
 False False False False False False False False False False False False
 False False False False False False  True False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False F

In [30]:

# Feature Selection with SHAP and RandomForest
rf_shap = RandomForestRegressor(n_estimators=100, random_state=42)
rf_shap.fit(X_test_scaled, y_test)
explainer = shap.TreeExplainer(rf_shap)
shap_values = explainer.shap_values(X_test_scaled)
shap_sum = np.abs(shap_values).mean(axis=0)
shap_features = shap_sum > np.percentile(shap_sum, 75)  # Top 25% SHAP values
print(shap_features)

[ True False False  True  True False  True  True  True  True  True  True
  True False False False False False False False False False False  True
 False  True  True False False  True  True False False False False False
 False False  True False False  True False False  True  True False False
  True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False  True]


In [31]:
from sklearn.svm import SVR

# Fit an SVR model
svr = SVR(kernel='linear')
svr.fit(X_train_scaled, y_train)

# Extract feature importance based on the coefficients
svr_features = np.abs(svr.coef_[0]) > np.percentile(np.abs(svr.coef_[0]), 75)  # Top 25% features
print("Features selected by SVR (Top 25%):", svr_features)

Features selected by SVR (Top 25%): [ True False  True  True  True False  True False False  True  True  True
  True False  True  True  True False  True False  True  True False False
 False False False False  True  True False  True False False False False
 False False False False False False False False  True  True False False
 False  True False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False]


In [ ]:
def evaluate_models(X_train, y_train, X_test, y_test):
    models = {
        "Linear Regression": LinearRegression(),
        "Lasso": Lasso(),
        "Random Forest": RandomForestRegressor(),
        "SVR": SVR(),
        "Gradient Boosting": GradientBoostingRegressor()
    }
    
    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        results[name] = rmse
        print(f"{name} RMSE: {rmse}")
    return results

In [32]:
def combine_feature_importances(X_columns, lasso_features, boruta_features, shap_features, svr_features, weights=None):
    if weights is None:
        weights = {'lasso': 1, 'boruta': 1, 'shap': 1, 'svr': 1}
    
    # Initialize scores array
    feature_scores = np.zeros(len(X_columns))

    # Assign scores based on feature selection outcomes
    feature_scores += weights['lasso'] * lasso_features
    feature_scores += weights['boruta'] * boruta_features
    feature_scores += weights['shap'] * shap_features
    feature_scores += weights['svr'] * svr_features

    # Normalize scores
    max_score = np.max(feature_scores)
    if max_score > 0:
        feature_scores /= max_score
    
    # Decide on a threshold to select features
    selected_features = feature_scores > 0.5  # Select features with a score above 0.5
   

    return selected_features

# Combine feature importances
combined_features = combine_feature_importances(
    X_columns=X_train.columns, 
    lasso_features=lasso_features,
    boruta_features=boruta_features,
    shap_features=shap_features,
    svr_features=svr_features
    #weights={'lasso': 1, 'boruta': 1.5, 'shap': 1.5, 'svr': 1}
)

print("Combined Feature Selection:")
print(X_train.columns[combined_features])

Combined Feature Selection:
Index(['Weekend', 'Year', 'Temperature', 'Feelslike', 'Windspeed',
       'Month_spline_1', 'Month_spline_2', 'Month_spline_3', 'Hour_spline_8',
       'Hour_spline_9', 'Hour_spline_23', 'Day of week_spline_0',
       'Working_hours'],
      dtype='object')


In [33]:
def cv_rmse(y_true, y_pred):
    """
    Calculate the Coefficient of Variation of Root Mean Square Error (CV(RMSE))
    :param y_true: array-like of true values
    :param y_pred: array-like of predicted values
    :return: CV(RMSE) value
    """
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    mean_y_true = np.mean(y_true)
    cv_rmse_value = rmse / mean_y_true
    return cv_rmse_value

def md_mape(y_true, y_pred):
    """
    Calculate the Median Absolute Percentage Error (MD(MAPE))
    :param y_true: array-like of true values
    :param y_pred: array-like of predicted values
    :return: MD(MAPE) value
    """
    ape = np.abs((y_true - y_pred) / y_true) * 100
    md_mape_value = np.median(ape)
    return md_mape_value

In [34]:
# Filter training and testing sets with combined features
X_train_selected = X_train.loc[:, combined_features]
X_test_selected = X_test.loc[:, combined_features]
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Model Training
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "SVR": SVR()
}

def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-10))

for model_name, model in models.items():
    model.fit(X_train_scaled, y_train)
    
    # Predictions
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)

    # Training Errors
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_smape = smape(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    train_cv_rmse = cv_rmse(y_train, y_train_pred)
    train_mdmape = md_mape(y_train, y_train_pred)

    # Testing Errors
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_smape = smape(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_cv_rmse = cv_rmse(y_test, y_test_pred)
    test_mdmape = md_mape(y_test, y_test_pred)

    # Normalized RMSE
    y_mean = np.mean(y_test)
    test_nrmse = test_rmse / y_mean

    print(f"{model_name}:")
    print(f"  Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
    print(f"  Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
    print(f"  Train SMAPE: {train_smape:.2f}%, Test SMAPE: {test_smape:.2f}%")
    print(f"  Train R^2: {train_r2:.4f}, Test R^2: {test_r2:.4f}")
    print(f"  Test NRMSE: {test_nrmse:.4f}")
    print(f"  Train CV RMSE: {train_cv_rmse:.4f}, Test CV RMSE: {test_cv_rmse:.4f}")
    print(f"  Train Median Mape: {train_mdmape:.4f}, Test Median Mape: {test_mdmape:.4f}")


Linear Regression:
  Train RMSE: 0.9452, Test RMSE: 0.8786
  Train MAE: 0.6953, Test MAE: 0.6255
  Train SMAPE: 37.23%, Test SMAPE: 29.11%
  Train R^2: 0.4344, Test R^2: 0.4329
  Test NRMSE: 0.4242
  Train CV RMSE: 0.5077, Test CV RMSE: 0.4242
  Train Median Mape: 32.7478, Test Median Mape: 24.6476
Random Forest:
  Train RMSE: 0.4511, Test RMSE: 0.8888
  Train MAE: 0.2882, Test MAE: 0.6040
  Train SMAPE: 15.67%, Test SMAPE: 26.90%
  Train R^2: 0.8712, Test R^2: 0.4196
  Test NRMSE: 0.4291
  Train CV RMSE: 0.2423, Test CV RMSE: 0.4291
  Train Median Mape: 11.5821, Test Median Mape: 20.8675
Gradient Boosting:
  Train RMSE: 0.8222, Test RMSE: 0.8152
  Train MAE: 0.5675, Test MAE: 0.5625
  Train SMAPE: 28.96%, Test SMAPE: 25.46%
  Train R^2: 0.5720, Test R^2: 0.5117
  Test NRMSE: 0.3936
  Train CV RMSE: 0.4417, Test CV RMSE: 0.3936
  Train Median Mape: 23.6837, Test Median Mape: 19.9886
Ridge:
  Train RMSE: 0.9452, Test RMSE: 0.8786
  Train MAE: 0.6953, Test MAE: 0.6255
  Train SMAPE: 37.2

In [35]:
# Filter training and testing sets with combined features
X_train_selected = X_train.loc[:, shap_features]
X_test_selected = X_test.loc[:, shap_features]
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Model Training
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "SVR": SVR()
}

def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-10))

for model_name, model in models.items():
    model.fit(X_train_scaled, y_train)
    
    # Predictions
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)

    # Training Errors
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_smape = smape(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    train_cv_rmse = cv_rmse(y_train, y_train_pred)
    train_mdmape = md_mape(y_train, y_train_pred)

    # Testing Errors
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_smape = smape(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_cv_rmse = cv_rmse(y_test, y_test_pred)
    test_mdmape = md_mape(y_test, y_test_pred)

    # Normalized RMSE
    y_mean = np.mean(y_test)
    test_nrmse = test_rmse / y_mean

    print(f"{model_name}:")
    print(f"  Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
    print(f"  Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
    print(f"  Train SMAPE: {train_smape:.2f}%, Test SMAPE: {test_smape:.2f}%")
    print(f"  Train R^2: {train_r2:.4f}, Test R^2: {test_r2:.4f}")
    print(f"  Test NRMSE: {test_nrmse:.4f}")
    print(f"  Train CV RMSE: {train_cv_rmse:.4f}, Test CV RMSE: {test_cv_rmse:.4f}")
    print(f"  Train Median Mape: {train_mdmape:.4f}, Test Median Mape: {test_mdmape:.4f}")


Linear Regression:
  Train RMSE: 0.9428, Test RMSE: 0.9005
  Train MAE: 0.7007, Test MAE: 0.6368
  Train SMAPE: 38.16%, Test SMAPE: 31.56%
  Train R^2: 0.4372, Test R^2: 0.4042
  Test NRMSE: 0.4347
  Train CV RMSE: 0.5065, Test CV RMSE: 0.4347
  Train Median Mape: 33.4636, Test Median Mape: 25.6347
Random Forest:
  Train RMSE: 0.2956, Test RMSE: 0.8954
  Train MAE: 0.1946, Test MAE: 0.5985
  Train SMAPE: 10.83%, Test SMAPE: 27.65%
  Train R^2: 0.9447, Test R^2: 0.4110
  Test NRMSE: 0.4322
  Train CV RMSE: 0.1588, Test CV RMSE: 0.4322
  Train Median Mape: 8.0973, Test Median Mape: 20.1879
Gradient Boosting:
  Train RMSE: 0.8147, Test RMSE: 0.8567
  Train MAE: 0.5704, Test MAE: 0.5764
  Train SMAPE: 29.55%, Test SMAPE: 26.80%
  Train R^2: 0.5798, Test R^2: 0.4607
  Test NRMSE: 0.4136
  Train CV RMSE: 0.4376, Test CV RMSE: 0.4136
  Train Median Mape: 24.3902, Test Median Mape: 19.6538
Ridge:
  Train RMSE: 0.9428, Test RMSE: 0.9005
  Train MAE: 0.7007, Test MAE: 0.6367
  Train SMAPE: 38.16

In [36]:
# Filter training and testing sets with combined features
X_train_selected = X_train.loc[:, lasso_features]
X_test_selected = X_test.loc[:, lasso_features]
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Model Training
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "SVR": SVR()
}

def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-10))

for model_name, model in models.items():
    model.fit(X_train_scaled, y_train)
    
    # Predictions
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)

    # Training Errors
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_smape = smape(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    train_cv_rmse = cv_rmse(y_train, y_train_pred)
    train_mdmape = md_mape(y_train, y_train_pred)

    # Testing Errors
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_smape = smape(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_cv_rmse = cv_rmse(y_test, y_test_pred)
    test_mdmape = md_mape(y_test, y_test_pred)

    # Normalized RMSE
    y_mean = np.mean(y_test)
    test_nrmse = test_rmse / y_mean

    print(f"{model_name}:")
    print(f"  Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
    print(f"  Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
    print(f"  Train SMAPE: {train_smape:.2f}%, Test SMAPE: {test_smape:.2f}%")
    print(f"  Train R^2: {train_r2:.4f}, Test R^2: {test_r2:.4f}")
    print(f"  Test NRMSE: {test_nrmse:.4f}")
    print(f"  Train CV RMSE: {train_cv_rmse:.4f}, Test CV RMSE: {test_cv_rmse:.4f}")
    print(f"  Train Median Mape: {train_mdmape:.4f}, Test Median Mape: {test_mdmape:.4f}")


Linear Regression:
  Train RMSE: 0.9150, Test RMSE: 0.8871
  Train MAE: 0.6801, Test MAE: 0.6365
  Train SMAPE: 37.60%, Test SMAPE: 31.84%
  Train R^2: 0.4699, Test R^2: 0.4218
  Test NRMSE: 0.4283
  Train CV RMSE: 0.4915, Test CV RMSE: 0.4283
  Train Median Mape: 32.4320, Test Median Mape: 26.0735
Random Forest:
  Train RMSE: 0.2490, Test RMSE: 0.8079
  Train MAE: 0.1627, Test MAE: 0.5477
  Train SMAPE: 9.03%, Test SMAPE: 24.58%
  Train R^2: 0.9607, Test R^2: 0.5204
  Test NRMSE: 0.3900
  Train CV RMSE: 0.1338, Test CV RMSE: 0.3900
  Train Median Mape: 6.8936, Test Median Mape: 19.6749
Gradient Boosting:
  Train RMSE: 0.7802, Test RMSE: 0.7958
  Train MAE: 0.5432, Test MAE: 0.5476
  Train SMAPE: 28.03%, Test SMAPE: 24.84%
  Train R^2: 0.6146, Test R^2: 0.5347
  Test NRMSE: 0.3842
  Train CV RMSE: 0.4191, Test CV RMSE: 0.3842
  Train Median Mape: 23.2679, Test Median Mape: 20.6124
Ridge:
  Train RMSE: 0.9150, Test RMSE: 0.8871
  Train MAE: 0.6800, Test MAE: 0.6365
  Train SMAPE: 37.60%

In [37]:
# Filter training and testing sets with combined features
X_train_selected = X_train.loc[:, boruta_features]
X_test_selected = X_test.loc[:, boruta_features]
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Model Training
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "SVR": SVR()
}

def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-10))

for model_name, model in models.items():
    model.fit(X_train_scaled, y_train)
    
    # Predictions
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)

    # Training Errors
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_smape = smape(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    train_cv_rmse = cv_rmse(y_train, y_train_pred)
    train_mdmape = md_mape(y_train, y_train_pred)

    # Testing Errors
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_smape = smape(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_cv_rmse = cv_rmse(y_test, y_test_pred)
    test_mdmape = md_mape(y_test, y_test_pred)

    # Normalized RMSE
    y_mean = np.mean(y_test)
    test_nrmse = test_rmse / y_mean

    print(f"{model_name}:")
    print(f"  Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
    print(f"  Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
    print(f"  Train SMAPE: {train_smape:.2f}%, Test SMAPE: {test_smape:.2f}%")
    print(f"  Train R^2: {train_r2:.4f}, Test R^2: {test_r2:.4f}")
    print(f"  Test NRMSE: {test_nrmse:.4f}")
    print(f"  Train CV RMSE: {train_cv_rmse:.4f}, Test CV RMSE: {test_cv_rmse:.4f}")
    print(f"  Train Median Mape: {train_mdmape:.4f}, Test Median Mape: {test_mdmape:.4f}")


Linear Regression:
  Train RMSE: 0.9634, Test RMSE: 0.9075
  Train MAE: 0.7128, Test MAE: 0.6386
  Train SMAPE: 38.17%, Test SMAPE: 29.39%
  Train R^2: 0.4124, Test R^2: 0.3950
  Test NRMSE: 0.4381
  Train CV RMSE: 0.5175, Test CV RMSE: 0.4381
  Train Median Mape: 34.0598, Test Median Mape: 24.4586
Random Forest:
  Train RMSE: 0.3695, Test RMSE: 0.8940
  Train MAE: 0.2377, Test MAE: 0.6096
  Train SMAPE: 13.04%, Test SMAPE: 27.15%
  Train R^2: 0.9135, Test R^2: 0.4128
  Test NRMSE: 0.4316
  Train CV RMSE: 0.1985, Test CV RMSE: 0.4316
  Train Median Mape: 9.6545, Test Median Mape: 21.0751
Gradient Boosting:
  Train RMSE: 0.8396, Test RMSE: 0.8425
  Train MAE: 0.5820, Test MAE: 0.5800
  Train SMAPE: 29.62%, Test SMAPE: 26.20%
  Train R^2: 0.5537, Test R^2: 0.4785
  Test NRMSE: 0.4067
  Train CV RMSE: 0.4510, Test CV RMSE: 0.4067
  Train Median Mape: 23.8728, Test Median Mape: 20.9511
Ridge:
  Train RMSE: 0.9634, Test RMSE: 0.9075
  Train MAE: 0.7128, Test MAE: 0.6386
  Train SMAPE: 38.17

In [38]:
# Filter training and testing sets with combined features
X_train_selected = X_train.loc[:, svr_features]
X_test_selected = X_test.loc[:, svr_features]
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Model Training
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "SVR": SVR()
}

def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-10))

for model_name, model in models.items():
    model.fit(X_train_scaled, y_train)
    
    # Predictions
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)

    # Training Errors
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_smape = smape(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    train_cv_rmse = cv_rmse(y_train, y_train_pred)
    train_mdmape = md_mape(y_train, y_train_pred)

    # Testing Errors
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_smape = smape(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_cv_rmse = cv_rmse(y_test, y_test_pred)
    test_mdmape = md_mape(y_test, y_test_pred)

    # Normalized RMSE
    y_mean = np.mean(y_test)
    test_nrmse = test_rmse / y_mean

    print(f"{model_name}:")
    print(f"  Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
    print(f"  Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
    print(f"  Train SMAPE: {train_smape:.2f}%, Test SMAPE: {test_smape:.2f}%")
    print(f"  Train R^2: {train_r2:.4f}, Test R^2: {test_r2:.4f}")
    print(f"  Test NRMSE: {test_nrmse:.4f}")
    print(f"  Train CV RMSE: {train_cv_rmse:.4f}, Test CV RMSE: {test_cv_rmse:.4f}")
    print(f"  Train Median Mape: {train_mdmape:.4f}, Test Median Mape: {test_mdmape:.4f}")


Linear Regression:
  Train RMSE: 0.9421, Test RMSE: 0.8894
  Train MAE: 0.6933, Test MAE: 0.6371
  Train SMAPE: 37.13%, Test SMAPE: 29.84%
  Train R^2: 0.4381, Test R^2: 0.4189
  Test NRMSE: 0.4294
  Train CV RMSE: 0.5061, Test CV RMSE: 0.4294
  Train Median Mape: 32.6684, Test Median Mape: 25.1301
Random Forest:
  Train RMSE: 0.3979, Test RMSE: 0.9065
  Train MAE: 0.2473, Test MAE: 0.6264
  Train SMAPE: 13.40%, Test SMAPE: 28.05%
  Train R^2: 0.8997, Test R^2: 0.3963
  Test NRMSE: 0.4376
  Train CV RMSE: 0.2138, Test CV RMSE: 0.4376
  Train Median Mape: 9.4323, Test Median Mape: 22.0308
Gradient Boosting:
  Train RMSE: 0.8253, Test RMSE: 0.8338
  Train MAE: 0.5763, Test MAE: 0.5795
  Train SMAPE: 29.65%, Test SMAPE: 26.25%
  Train R^2: 0.5687, Test R^2: 0.4892
  Test NRMSE: 0.4025
  Train CV RMSE: 0.4434, Test CV RMSE: 0.4025
  Train Median Mape: 24.8205, Test Median Mape: 21.6146
Ridge:
  Train RMSE: 0.9421, Test RMSE: 0.8894
  Train MAE: 0.6933, Test MAE: 0.6371
  Train SMAPE: 37.13